In [4]:
from dotenv import load_dotenv

In [5]:
load_dotenv()

True

In [6]:
# %pip install langchain-groq
import os

# os.getenv("GROQ_API_KEY")

In [13]:
# from langchain.tools import tool
# from langchain_core.messages import HumanMessage
# from langgraph.checkpoint.memory import MemorySaver
# from langgraph.prebuilt import create_react_agent


from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_groq import ChatGroq

from langchain_core.output_parsers import JsonOutputParser

In [24]:
from typing import List
from langchain_core.pydantic_v1 import BaseModel, Field
import time


class ObjectsDetected(BaseModel):
    objects_detected: List[str] = Field(description="List of detected objects")

        
llm = ChatGroq(model="llama3-70b-8192", temperature=0.5)
template = '''
You are an advanced language model tasked with merging detailed captions generated by two multimodal systems. Your goal is to create a unified, highly detailed, and factually accurate description of the image.  

Below is a detailed description of an image. Your task is to extract and list every object mentioned in the description that can be reliably detected by an object detection model. Please follow these instructions carefully:  

### Instructions:  
1. Identify each object that is clearly described and can be recognized by typical object detection systems.  
2. Be very cautious with numerical details attached to the objects, don't avoid them.  
3. Do not combine or generalize objects if different numbers are specified. Each unique instance should be explicitly listed.  
'''
system_message = SystemMessagePromptTemplate.from_template(template)
human_message = HumanMessagePromptTemplate.from_template("Here is description: {description}")

prompt_object_detection = ChatPromptTemplate.from_messages([system_message, human_message])
model = prompt_object_detection | llm.with_structured_output(ObjectsDetected)


In [26]:
request_counter = 0
last_reset_time = time.time()


def invoke_with_rate_limit_per_minute(description, requests_per_minute = 29):
    global request_counter, last_reset_time

    current_time = time.time()
    if current_time - last_reset_time >= 60:
        request_counter = 0
        last_reset_time = current_time

    if request_counter >= requests_per_minute:
        wait_time = time_window - (current_time - last_reset_time)
        print(f"Rate limit reached. Please wait {wait_time:.2f} seconds.")
        time.sleep(wait_time)  # Sleep until the next allowed time
        request_counter = 0  # Reset the counter after waiting
        last_reset_time = time.time()

    # Now make the request
    response = model.invoke({"description": description}).objects_detected

    # Increment the request counter after the request
    request_counter += 1

    return response

# Example usage
result = invoke_with_rate_limit_per_minute("there are 3 cars are driving on dirty route")
print(result)


['car', 'car', 'car', 'route']


In [ ]:
# TODO: regenerate first_llama_responses with groq 80b llama